In [5]:
import pandas as pd
from pyworkforce.queuing import MultiErlangC, ErlangC
import streamlit as st
import matplotlib.pyplot as plt

In [3]:
st.cache_data()
def compute_staffing_levels(demand, aht, planning_period, asa, shrinkage, service_level, max_occupancy):
   
    param_grid = {
        "transactions": demand.Predictions, 
        "aht": aht, 
        "interval": planning_period, 
        "asa": asa, 
        "shrinkage": shrinkage        
    }
   
    multi_erlang = MultiErlangC(param_grid=param_grid, n_jobs=-1)
    required_positions_scenarios = {"service_level": service_level, "max_occupancy": max_occupancy}
    positions_requirements = multi_erlang.required_positions(required_positions_scenarios)
    df = pd.DataFrame.from_dict(positions_requirements)
    df.index = demand.index
    staffing_levels = pd.concat([demand, df], axis=1)
    
    staffing_levels.columns = ["Number of Calls", "Raw Staffing Level", "Staffing Level",
                               "Service Level (%)", "Occupancy (%)", "Waiting Probability (%)"]
    
    percentage_columns = ["Service Level (%)", "Occupancy (%)", "Waiting Probability (%)"]
    
    pd.options.display.float_format = '{:,.2f}'.format
    
    staffing_levels[percentage_columns] = staffing_levels[percentage_columns].apply(lambda x: x*100)
    staffing_levels = staffing_levels.round(2)
#     staffing_levels["call_type"] = Type
    return staffing_levels



def plot_staffing_requirements(demand, params,t):
    q = 60//t
    demand["how"] = demand.reset_index(inplace = False).index
    axs = plt.subplot()
    y_max = demand.Staffing_level.max() +2 
    y_min = max(0, demand.Staffing_level.min() -2) 
    plt.plot(demand["how"], demand["Staffing_level"],zorder = 2, color = "green")
    for day in range(0,(24*7*q), 24*q):
            axs.vlines(x = day, ymin = 0, ymax =y_max + 0.2, color = "blue", zorder = 1, alpha = .4, linestyles= "dashed")
    # axs.hlines(y = 3, xmin=demand["how"].min(), xmax = demand["how"].max(), color = "tomato", zorder = 2)
    axs.set_xticks(list(range(1,(24*7*q)+1, 24*q)))
    xticks_minor = list(range(0 ,(24*7*q)+1,q))
    axs.set_xticks(xticks_minor, minor=True)
    axs.set_xticklabels(["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
    axs.set_xlim(xmin= 0, xmax = 24*7*q)
    axs.set_ylim(ymin= 1, ymax = y_max -1)
    axs.set_facecolor("lightgray")
    axs.set_ylabel("Number of employees")
    axs.set_xlabel("Average Week")
    plt.show()
#     print("Call Type:", call_type)
#     for p in params.keys():
#             text = f"{p.title()}: {params[p][call_type]}"
#             if p in {"asa", "aht"}:
#                     text += " seconds"
#             elif p in {"max_occupancy", "shrinkage", "service_level"}:
#                     text += "%"
#             print(text)


## Testing

In [60]:
def compute_staffing_levels(demand, aht, asa, interval, shrinkage, service_level, max_occupancy):
    demand = demand
    aht = aht
    asa = asa
    interval = interval
    shrinkage = shrinkage
    service_level = service_level
    max_occupancy = max_occupancy


    erlangC_formula = ErlangC(demand, aht, asa, interval, shrinkage)
    positions_requirements = erlangC_formula.required_positions(service_level=service_level, max_occupancy=max_occupancy)
    df = pd.DataFrame(positions_requirements, index=[0])
    staffing_levels = df
    staffing_levels['Number of surgeries'] = demand
    staffing_levels.columns = ["Raw Staffing Level", "Staffing Level",
                                "Service Level (%)", "Occupancy (%)", "Waiting Probability (%)", "Number of Surgeries"]

    percentage_columns = ["Service Level (%)", "Occupancy (%)", "Waiting Probability (%)"]
    pd.options.display.float_format = '{:,.2f}'.format
    staffing_levels[percentage_columns] = staffing_levels[percentage_columns].apply(lambda x: x*100)
    staffing_levels = staffing_levels.round(2)
    return staffing_levels

In [63]:
# DEFAULT SCENARIO PARAMETERS - OK 
params = {"service_level" : 0.9,
        "shrinkage" :       0.4,
        "max_occupancy" :   0.8,
        "asa" :             8,
        "aht":              2,
        'interval':         8}

interval = value = params['interval']
service_level = value= params["service_level"]
asa = value = params["asa"]
aht = value = params["aht"]
shrinkage = value = params["shrinkage"]
max_occupancy = value = params["max_occupancy"]
demand = int(1100/365)


staffing_levels = compute_staffing_levels(demand = demand,
                                          aht = aht,
                                          asa=asa,
                                          interval=interval,
                                          shrinkage=shrinkage,
                                          service_level=service_level,
                                          max_occupancy=max_occupancy)